# DB 4Houre Signal Scheduler

In [4]:
import pandas as pd
import numpy as np
import sqlite3
import time
import schedule
from binance.client import Client
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from datetime import datetime
from prophet import Prophet
import json
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# API keys
api_key = 'dd1a2703da01c24ccdbe10b2cbc9600a6218c41434d4453916766e1e817aabfe'
api_secret = '2acb5c37db3c5646fdadb366cacb5d9a11ffe05d7b4d01dfb1b3047f2a2ea09a'

# Initialize Binance client
client = Client(api_key, api_secret, testnet=True)

# Define LSTM model architecture
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define GRU model architecture
def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(128, activation='relu', return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(GRU(64, activation='relu', return_sequences=False))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae')
    return model

# Load pre-trained models
def load_models():
    try:
        lstm_model = create_lstm_model((20, 1))
        lstm_model.load_weights('lstmw.weights.h5')
        print("LSTM model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading LSTM model weights: {e}")
        lstm_model = None

    try:
        gru_model = create_gru_model((10, 1))
        gru_model.load_weights('model.weights.h5')
        print("GRU model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading GRU model weights: {e}")
        gru_model = None

    try:
        prophet_model = load_prophet_model('prophet_model.json')
        print("Prophet model loaded successfully.")
    except Exception as e:
        print(f"Error loading Prophet model: {e}")
        prophet_model = None

    return lstm_model, gru_model, prophet_model

def load_prophet_model(json_file):
    with open(json_file, 'r') as f:
        model_json = f.read()
    model = Prophet()
    model.__dict__.update(json.loads(model_json))
    return model

# Fetch data from Binance
def fetch_binance_data(start_date, end_date):
    klines = client.futures_klines(symbol='BTCUSDT', interval='1m', start_str=start_date, end_str=end_date)
    df = pd.DataFrame(klines, columns=['date_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['date_time'] = pd.to_datetime(df['date_time'], unit='ms')
    df.set_index('date_time', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

# Prepare sequences for model prediction
def prepare_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

# Calculate indicators
def calculate_super_trend(data, atr_period=10, multiplier=3):
    data['H-L'] = data['high'] - data['low']
    data['H-PC'] = np.abs(data['high'] - data['close'].shift(1))
    data['L-PC'] = np.abs(data['low'] - data['close'].shift(1))
    data['TR'] = data[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    data['ATR'] = data['TR'].rolling(window=atr_period).mean()
    data['Upper Band'] = (data['high'] + data['low']) / 2 + (multiplier * data['ATR'])
    data['Lower Band'] = (data['high'] + data['low']) / 2 - (multiplier * data['ATR'])
    data['Super_Trend'] = np.where(data['close'] > data['Upper Band'], 1, -1)
    return data['Super_Trend'].iloc[-1]

def calculate_ema_signal(data, period=14):
    data['EMA'] = data['close'].ewm(span=period, adjust=False).mean()
    return 1 if data['close'].iloc[-1] > data['EMA'].iloc[-1] else -1

def calculate_stochastic_signal(data, period=14):
    data['Low_Min'] = data['low'].rolling(window=period).min()
    data['High_Max'] = data['high'].rolling(window=period).max()
    data['%K'] = (data['close'] - data['Low_Min']) * 100 / (data['High_Max'] - data['Low_Min'])
    data['%D'] = data['%K'].rolling(window=3).mean()
    return 1 if data['%K'].iloc[-1] > data['%D'].iloc[-1] else -1

def calculate_bollinger_bands_signal(data, period=20, std_dev=2):
    data['SMA'] = data['close'].rolling(window=period).mean()
    data['Upper_Band'] = data['SMA'] + std_dev * data['close'].rolling(window=period).std()
    data['Lower_Band'] = data['SMA'] - std_dev * data['close'].rolling(window=period).std()
    return 1 if data['close'].iloc[-1] > data['Upper_Band'].iloc[-1] else -1

def calculate_hybrid_signal(data):
    super_trend_signal = calculate_super_trend(data)
    ema_signal = calculate_ema_signal(data)
    return 1 if super_trend_signal == 1 and ema_signal == 1 else -1

# Generate model-based signals
def generate_model_signals(df, lstm_model, gru_model, prophet_model):
    close_prices = df['close'].values
    sequences = prepare_sequences(close_prices, sequence_length=20)

    lstm_signal = np.zeros(len(close_prices))
    gru_signal = np.zeros(len(close_prices))
    prophet_signal = np.zeros(len(close_prices))

    if lstm_model:
        try:
            lstm_pred = lstm_model.predict(sequences)
            lstm_signal = np.where(lstm_pred > df['close'].iloc[-len(lstm_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating LSTM signals: {e}")

    if gru_model:
        try:
            gru_pred = gru_model.predict(sequences)
            gru_signal = np.where(gru_pred > df['close'].iloc[-len(gru_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating GRU signals: {e}")

    if prophet_model:
        try:
            future_dates = pd.DataFrame(df.index, columns=['ds'])
            prophet_forecast = prophet_model.predict(future_dates)
            prophet_signal = np.where(prophet_forecast['yhat'] > df['close'].values, 1, -1)
        except Exception as e:
            print(f"Error generating Prophet signals: {e}")

    return pd.DataFrame({
        'LSTM_Signals': lstm_signal,
        'GRU_Signals': gru_signal,
        'FB_signal': prophet_signal
    }, index=df.index)

# Append signals to the SQL database
def append_signals_to_db(signals_df):
    conn = sqlite3.connect('indicators.db')
    signals_df.to_sql('indicators', conn, if_exists='append', index=True)
    conn.close()

# Main process
def run_process():
    lstm_model, gru_model, prophet_model = load_models()

    start_date = '2024-08-12 00:00:00'
    end_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    df = fetch_binance_data(start_date, end_date)

    # Generate indicator-based signals
    super_trend_signal = calculate_super_trend(df)
    ema_signal = calculate_ema_signal(df)
    stochastic_signal = calculate_stochastic_signal(df)
    bollinger_signal = calculate_bollinger_bands_signal(df)
    hybrid_signal = calculate_hybrid_signal(df)

    # Generate model-based signals
    model_signals = generate_model_signals(df, lstm_model, gru_model, prophet_model)
    
    # Combine all signals into one DataFrame
    all_signals_df = pd.concat([model_signals, 
                                pd.DataFrame({'Super_Trend': super_trend_signal, 'EMA_Signal': ema_signal, 
                                              'Stochastic_Signal': stochastic_signal, 'Bollinger_Signal': bollinger_signal,
                                              'Hybrid_Signal': hybrid_signal}, index=df.index)], axis=1)

    # Save to CSV file
    all_signals_df.to_csv('combined_signals.csv')

    # Append to SQLite DB
    append_signals_to_db(all_signals_df)

# Schedule the script to run every 4 hours
schedule.every(4).hours.do(run_process)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)


c:\Users\MTariq\Downloads\python-binance-master\Desktop\python-binance-master\tests\.conda\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\MTariq\Downloads\python-binance-master\Desktop\python-binance-master\tests\.conda\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


LSTM model weights loaded successfully.


c:\Users\MTariq\Downloads\python-binance-master\Desktop\python-binance-master\tests\.conda\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Error loading GRU model weights: A total of 3 objects could not be loaded. Example error message for object <GRUCell name=gru_cell, built=True>:

Layer 'gru_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<GRUCell name=gru_cell, built=True>, <GRUCell name=gru_cell, built=True>, <Dense name=dense_7, built=True>]
Prophet model loaded successfully.
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
Error generating Prophet signals: Dataframe has no rows.


ValueError: Data must be 1-dimensional, got ndarray of shape (480, 480) instead

# Delete Some Rows

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input
from prophet import Prophet
import schedule
import time

# Model Definitions
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(128, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64, activation='relu', return_sequences=False))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae')
    return model

# Load Models
def load_models():
    try:
        lstm_model = create_lstm_model((20, 1))
        lstm_model.load_weights('lstmw.weights.h5')
        print("LSTM model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading LSTM model weights: {e}")
        lstm_model = None

    try:
        gru_model = create_gru_model((10, 1))
        gru_model.load_weights('gru_model.weights.h5')
        print("GRU model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading GRU model weights: {e}")
        gru_model = None

    try:
        prophet_model = Prophet()
        prophet_model.load('prophet_model.pkl')
        print("Prophet model loaded successfully.")
    except Exception as e:
        print(f"Error loading Prophet model: {e}")
        prophet_model = None

    return lstm_model, gru_model, prophet_model

# Prepare Sequences
def prepare_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences).reshape(-1, sequence_length, 1)

# Generate Model Signals
def generate_model_signals(df, lstm_model, gru_model, prophet_model):
    close_prices = df['close'].values
    sequences = prepare_sequences(close_prices, sequence_length=20)

    lstm_signal = np.zeros(len(close_prices))
    gru_signal = np.zeros(len(close_prices))
    prophet_signal = np.zeros(len(close_prices))

    if lstm_model:
        try:
            lstm_pred = lstm_model.predict(sequences)
            lstm_signal = np.where(lstm_pred > df['close'].iloc[-len(lstm_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating LSTM signals: {e}")

    if gru_model:
        try:
            gru_pred = gru_model.predict(sequences)
            gru_signal = np.where(gru_pred > df['close'].iloc[-len(gru_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating GRU signals: {e}")

    if prophet_model:
        try:
            future_dates = pd.DataFrame(df.index, columns=['ds'])
            prophet_forecast = prophet_model.predict(future_dates)
            prophet_signal = np.where(prophet_forecast['yhat'] > df['close'].values, 1, -1)
        except Exception as e:
            print(f"Error generating Prophet signals: {e}")

    return pd.DataFrame({
        'LSTM_Signals': lstm_signal,
        'GRU_Signals': gru_signal,
        'FB_signal': prophet_signal
    }, index=df.index)

# Scheduler Task
def job():
    df = pd.read_csv('data.csv')  # Replace with actual data loading
    lstm_model, gru_model, prophet_model = load_models()
    if lstm_model is not None and gru_model is not None and prophet_model is not None:
        signals_df = generate_model_signals(df, lstm_model, gru_model, prophet_model)
        signals_df.to_csv('model_signals.csv')
    else:
        print("Not all models are loaded successfully.")

# Schedule Task
schedule.every().hour.do(job)

# Run the Scheduler
while True:
    schedule.run_pending()
    time.sleep(1)


c:\Users\MTariq\Downloads\python-binance-master\Desktop\python-binance-master\tests\.conda\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


LSTM model weights loaded successfully.
Error loading GRU model weights: [Errno 2] Unable to synchronously open file (unable to open file: name = 'gru_model.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Error loading Prophet model: 'Prophet' object has no attribute 'load'
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


ValueError: Data must be 1-dimensional, got ndarray of shape (480, 480) instead

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input
from prophet import Prophet
import schedule
import time

# Model Definitions
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(128, activation='relu', return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(64, activation='relu', return_sequences=False))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mae')
    return model

# Load Models
def load_models():
    try:
        lstm_model = create_lstm_model((20, 1))
        lstm_model.load_weights('lstmw.weights.h5')
        print("LSTM model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading LSTM model weights: {e}")
        lstm_model = None

    try:
        gru_model = create_gru_model((10, 1))
        gru_model.load_weights('gru_model.weights.h5')
        print("GRU model weights loaded successfully.")
    except Exception as e:
        print(f"Error loading GRU model weights: {e}")
        gru_model = None

    try:
        prophet_model = Prophet()
        prophet_model.load('prophet_model.pkl')
        print("Prophet model loaded successfully.")
    except Exception as e:
        print(f"Error loading Prophet model: {e}")
        prophet_model = None

    return lstm_model, gru_model, prophet_model

# Prepare Sequences
def prepare_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences).reshape(-1, sequence_length, 1)

# Generate Model Signals
def generate_model_signals(df, lstm_model, gru_model, prophet_model):
    close_prices = df['close'].values
    sequences = prepare_sequences(close_prices, sequence_length=20)

    lstm_signal = np.zeros(len(close_prices))
    gru_signal = np.zeros(len(close_prices))
    prophet_signal = np.zeros(len(close_prices))

    if lstm_model:
        try:
            lstm_pred = lstm_model.predict(sequences)
            lstm_signal = np.where(lstm_pred > df['close'].iloc[-len(lstm_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating LSTM signals: {e}")

    if gru_model:
        try:
            gru_pred = gru_model.predict(sequences)
            gru_signal = np.where(gru_pred > df['close'].iloc[-len(gru_pred):].values, 1, -1)
        except Exception as e:
            print(f"Error generating GRU signals: {e}")

    if prophet_model:
        try:
            future_dates = pd.DataFrame(df.index, columns=['ds'])
            prophet_forecast = prophet_model.predict(future_dates)
            prophet_signal = np.where(prophet_forecast['yhat'] > df['close'].values, 1, -1)
        except Exception as e:
            print(f"Error generating Prophet signals: {e}")

    return pd.DataFrame({
        'LSTM_Signals': lstm_signal,
        'GRU_Signals': gru_signal,
        'FB_signal': prophet_signal
    }, index=df.index)

# Scheduler Task
def job():
    df = pd.read_csv('data.csv')  # Replace with actual data loading
    lstm_model, gru_model, prophet_model = load_models()
    if lstm_model is not None and gru_model is not None and prophet_model is not None:
        signals_df = generate_model_signals(df, lstm_model, gru_model, prophet_model)
        signals_df.to_csv('model_signals.csv')
    else:
        print("Not all models are loaded successfully.")

# Schedule Task
schedule.every().hour.do(job)

# Run the Scheduler
while True:
    schedule.run_pending()
    time.sleep(1)


c:\Users\MTariq\Downloads\python-binance-master\Desktop\python-binance-master\tests\.conda\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


LSTM model weights loaded successfully.
Error loading GRU model weights: [Errno 2] Unable to synchronously open file (unable to open file: name = 'gru_model.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)
Error loading Prophet model: 'Prophet' object has no attribute 'load'
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


ValueError: Data must be 1-dimensional, got ndarray of shape (480, 480) instead

In [48]:
import schedule
import time

# Scheduler to run every 4 hours
schedule.every(4).hours.do(run_process)

# Run the process
while True:
    schedule.run_pending()
    time.sleep(1)


TypeError: generate_model_signals() missing 3 required positional arguments: 'lstm_model', 'gru_model', and 'prophet_model'

In [46]:
def generate_model_signals(df, lstm_model, gru_model, prophet_model):
    close_prices = df['close'].values
    sequences = prepare_sequences(close_prices)

    # Initialize signal arrays
    lstm_signal = np.zeros(len(close_prices))
    gru_signal = np.zeros(len(close_prices))
    prophet_signal = np.zeros(len(close_prices))

    # Generate LSTM model signals
    if lstm_model:
        try:
            # Predict using LSTM
            lstm_pred = lstm_model.predict(sequences)
            # Generate signals based on prediction
            lstm_signal[len(lstm_pred):] = np.where(lstm_pred > close_prices[:len(lstm_pred)], 1, -1)
        except Exception as e:
            print(f"Error generating LSTM signals: {e}")

    # Generate GRU model signals
    if gru_model:
        try:
            # Predict using GRU
            gru_pred = gru_model.predict(sequences)
            # Generate signals based on prediction
            gru_signal[len(gru_pred):] = np.where(gru_pred > close_prices[:len(gru_pred)], 1, -1)
        except Exception as e:
            print(f"Error generating GRU signals: {e}")

    # Generate Prophet model signals
    if prophet_model:
        try:
            # Prepare data for Prophet
            future_dates = pd.DataFrame(df.index, columns=['ds'])
            prophet_forecast = prophet_model.predict(future_dates)
            # Generate signals based on prediction
            prophet_signal = np.where(prophet_forecast['yhat'] > close_prices, 1, -1)
        except Exception as e:
            print(f"Error generating Prophet signals: {e}")

    return pd.DataFrame({
        'LSTM_Signals': lstm_signal,
        'GRU_Signals': gru_signal,
        'FB_signal': prophet_signal
    }, index=df.index)


In [40]:
from prophet import Prophet
import json

def load_prophet_model(json_file):
    with open(json_file, 'r') as f:
        model_json = f.read()
    model = Prophet()
    # Load model configuration here
    return model

try:
    prophet_model = load_prophet_model('prophet_model.json')
    print("Prophet model loaded successfully.")
except Exception as e:
    print(f"Error loading Prophet model: {e}")


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:TBB already found in load path


Prophet model loaded successfully.
